## <span style="color:yellow; font-weight:bold; font-family:Times, sans-serif">Library Seekers</span>

#### <span style="color:yellow; font-weight:bold; font-family:Times, sans-serif">Introdução</span>

Foram utilizadas duas bases de dados, a primeira contendo informações a respeito de diversos livros e a segunda trazendo informações fornecidas por leitores desses livros, avaliando-os

Neste Jupyter Notebook, em primeiro momento foram realizados tratamentos nas duas bases de dados utilizadas, a fim de retirar caracteres poluentes, uniformizar formato de data, agrupamentos para junção de informações, dentre outras. Em seguida foi utilzado um modelo NLP com o objetivo de analisar o sentimento do resumo das avaliações dos leitores e um outro modelo para resumir a descrição (pode ser considerada uma sinopse) dos livros.

Na análise exploratória foram plotados alguns gráficos trazendo ranqueamentos dos livros, das editoras, dos autores e das categorias, destacando o top5 de cada um dos rankings

#### <span style="color:yellow; font-weight:bold; font-family:Times, sans-serif">Instalação</span>
Para executar este notebook, você precisará das seguintes bibliotecas e dependências:

```python
!pip install pandas
!pip install seaborn
!pip install matplotlib
!pip install tqdm
!pip install transformers
!pip install tensorflow
!pip install tf-keras
!pip install ipywidgets
!pip install plotly

### 1.0 - Importações e configurações

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

from transformers import pipeline
import plotly.graph_objects as go

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### 2.0 - Bases

In [9]:
bd = pd.read_parquet(r'..\\..\\Desafio_A3Data\\Databases\\books_data.parquet', engine='pyarrow')
br = pd.read_parquet(r'..\\..\\Desafio_A3Data\\Databases\\books_rating.parquet', engine='pyarrow')

### 3.0 - Tratamentos

In [10]:
#-----------------------BOOKS RATING------------------------------

br['summary'].fillna('No summary', inplace=True)
br['text'].fillna('No text comments', inplace=True)

#Realizando agrupamento pelo título do livro e obtendo a média dos valores numéricos
temporary = br[['Title', 'Price', 'score', 'time']]
temporary = temporary.groupby('Title').mean()
temporary.reset_index(inplace=True)

#Identificando quantas avaliações cada livro recebeu pelos leitores
temporary2 = br.groupby('Title').size().reset_index(name='readersRatings')


#-----------------------BOOKS DATA--------------------------------

#Adicionando informações capturadas acima ao dataset dos livros
merge = bd.merge(temporary, on='Title', how='inner')
merge2 = merge.merge(temporary2, on='Title', how='inner')
bd = merge2

#Retirando caracteres poluentes das duas variáveis abaixo
bd['categories'] = bd['categories'].str.replace('[\[\]\'\"]', '', regex=True)
bd['authors'] = bd['authors'].str.replace('[\[\]\'\"]', '', regex=True)

#Reservando somente o ano de publicação, dado que não havia uma uniformidade 
#no formato da data, alguns somente com o ano, outros no formato yyyy/mm/dd
bd['publishedDate'] = bd['publishedDate'].str[:4]

#Obtendo totais de avaliações para cada livro
bd['ratingsCount'].fillna(0, inplace=True)
bd['totalRatings'] = bd['readersRatings'] + bd['ratingsCount']

bd.dropna(subset=['description'], inplace=True)

In [44]:
#Filtro realizado devido quantidade de cometários no dataset books_rating ser muito grande, 
#o que ocasionou uma limitação no tempo de procesamento dos campos textuais
#Retirando essa célula do processamento, o fluxo ocorre normalmente para o dataset completo
bd = bd[bd['readersRatings'] == 100]
selected_books = bd['Title'].unique()

br = br[br['Title'].isin(selected_books)]

In [13]:
#Modelo de captação de sentimento usado
sentiment_classifier = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", 
    return_all_scores=True
)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
c:\lib\site-packages\transformers\pipelines\text_classification.py:104: UserWarning:

`return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.



In [14]:
text = br['summary'][66533]
text

'Man vs Beast and the Birth of Civilization'

In [15]:
result = sentiment_classifier(text)
result

[[{'label': 'positive', 'score': 0.531891942024231},
  {'label': 'neutral', 'score': 0.18506908416748047},
  {'label': 'negative', 'score': 0.2830389142036438}]]

In [ ]:
# Criando uma lista vazia para armazenar os resultados
positive = []
neutral = []
negative = []

# Iterando sobre cada valor da coluna 'summary'
for value in tqdm(br['summary'], desc='Progress'):
    
    # Aplicando a função a cada valor
    result = sentiment_classifier(value)

    #Obtendo os valores dos resultados
    positive_value = result[0][0]['score']
    neutral_value = result[0][1]['score']
    negative_value = result[0][2]['score']

    # Adicionando os resultados à lista
    positive.append(positive_value)
    neutral.append(neutral_value)
    negative.append(negative_value)

# Criando novas colunas no DataFrame com os resultados
br['summary_posit_score'] = positive
br['summary_neut_score'] = neutral
br['summary_neg_score'] = negative

In [17]:
#Função utilizada para obter resumo dos scores gerados pela função sentiment_classifier
def categorize(row):
    if (row['summary_posit_score'] + row['summary_neut_score']) > row['summary_neg_score']:
        return 'positivo'
    else:
        return 'negativo'

# Aplicando a função a cada linha do DataFrame
br['summary_score'] = br.apply(categorize, axis=1)

In [ ]:
summarizer = pipeline("summarization", model="Falconsai/text_summarization")

In [ ]:
# Criando uma lista vazia para armazenar os resultados
summary = []

# Iterando sobre cada valor da coluna 'description'
for value in tqdm(bd['description'], desc='Progress'):
    # Aplicando a função a cada valor
    result = summarizer(value)

    summary_value = result[0]['summary_text']

    # Adicionando o resultado à lista
    summary.append(summary_value)
    
# Criando uma nova coluna no DataFrame com os resultados
bd['summaryDescription'] = summary

In [48]:
print(bd.shape)
bd.head(5)

(38, 16)


,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount,Price,score,time,readersRatings,totalRatings,summaryDescription
4553,Death in the Afternoon,Still considered one of the best books ever wr...,Ernest Hemingway,http://books.google.com/books/content?id=AdFQA...,http://books.google.nl/books?id=AdFQAQAAQBAJ&p...,Simon and Schuster,2014,https://play.google.com/store/books/details?id...,Literary Criticism,9.0,NaN,3.96,308.96400,100,109.0,Death in the Afternoon is an impassioned look ...
9230,The Geographer's Library,Item 1: An alembic is the top part of an appar...,Jon Fasman,http://books.google.com/books/content?id=H5ogA...,http://books.google.nl/books?id=H5ogAQAAIAAJ&q...,Penguin Press HC,2005,http://books.google.nl/books?id=H5ogAQAAIAAJ&d...,Fiction,33.0,NaN,3.06,324.26232,100,133.0,Item 1: An alembic is the top part of an appar...
13509,LifeSupport,From New York Times bestselling author Tess Ge...,Tess Gerritsen,http://books.google.com/books/content?id=s-bKp...,http://books.google.nl/books?id=s-bKpPSlbbUC&q...,Pocket Books,1998,http://books.google.nl/books?id=s-bKpPSlbbUC&d...,Fiction,2.0,NaN,4.20,310.63536,100,102.0,The quiet overnight shift at Springer Hospital...
14923,Population: 485 : Meeting Your Neighbors One S...,"Welcome to New Auburn, Wisconsin, where the lo...",Michael Perry,http://books.google.com/books/content?id=IOhWr...,http://books.google.nl/books?id=IOhWrgEACAAJ&d...,Harper Perennial,2007,http://books.google.nl/books?id=IOhWrgEACAAJ&d...,Travel,25.0,NaN,4.62,326.88408,100,125.0,The most senior member of the volunteer fire d...
21679,The Light of Western Stars,This eBook features the unabridged text of ‘Th...,Zane Grey,http://books.google.com/books/content?id=EKDWD...,http://books.google.com/books?id=EKDWDwAAQBAJ&...,Delphi Classics,2017,https://play.google.com/store/books/details?id...,Fiction,0.0,25.8,4.38,374.31096,100,100.0,This eBook features the unabridged text of ‘Th...


In [49]:
print(br.shape)
br.head(5)

(3800, 13)


,Id,Title,Price,User_id,profileName,score,time,summary,text,summary_posit_score,summary_neut_score,summary_neg_score,summary_score
66533,B000JWW1UG,Death in the Afternoon,NaN,A1ZA12IECEKIY,Mark Cohen,4.0,376.680,Man vs Beast and the Birth of Civilization,Bullfighting was born in Spain many centuries ...,0.531892,0.185069,0.283039,positivo
66534,B000JWW1UG,Death in the Afternoon,NaN,A2UO3C0HLKCZ4F,R. Huitron,4.0,376.320,if one is born outside of bullfighting culture...,A lot of times people end up hating something ...,0.190422,0.166635,0.642942,negativo
66535,B000JWW1UG,Death in the Afternoon,NaN,AIHBELNQRGPVD,acacia,4.0,375.024,death in the afternoon,I enjoyed the first part of the book all about...,0.095906,0.146184,0.757909,negativo
66536,B000JWW1UG,Death in the Afternoon,NaN,A1N7T3TGQT3CS3,diogenes,4.0,373.608,Excellent primer for bullfighting lore.,I've reread this book for the third time this ...,0.893192,0.067662,0.039146,positivo
66537,B000JWW1UG,Death in the Afternoon,NaN,A2WXFSE5PJ70NV,Jim,5.0,372.816,Classic,Excellent book about bullfighting and travelli...,0.637950,0.234253,0.127797,positivo


In [ ]:
bd['time'] = bd['time'] / 3600000
br['time'] = br['time'] / 3600000

In [46]:
bd.to_parquet(r'..\\..\\Desafio_A3Data\\Databases\\work_books_data.parquet', engine='pyarrow')
br.to_parquet(r'..\\..\\Desafio_A3Data\\Databases\\work_books_rating.parquet', engine='pyarrow')

### 4.0 - EDA

In [4]:
bd = pd.read_parquet(r'..\\..\\Desafio_A3Data\\Databases\\work_books_data.parquet', engine='pyarrow')
br = pd.read_parquet(r'..\\..\\Desafio_A3Data\\Databases\\work_books_rating.parquet', engine='pyarrow')

In [51]:
# Ordenar os dados pelo score do maior para o menor
bd_sorted = bd.sort_values(by='score', ascending=False)

# Dados para o gráfico
categorias = bd_sorted['Title']
valores = bd_sorted['score']

# Criação do gráfico de barras horizontal
fig = go.Figure(data=[
    go.Bar(
        name='Valores', 
        y=categorias, 
        x=valores, 
        orientation='h',
        marker=dict(
            color='#1f77b4'  # Cor das barras
        ),
        text=valores,  # Adiciona os valores como rótulos
        textposition='outside',  # Posição dos rótulos
        hoverinfo='y+x'  # Informação exibida ao passar o mouse
    )
])

# Personalização do layout do gráfico
fig.update_layout(
    title='Scores dos Livros: Avaliação dos Leitores',
    title_font=dict(size=20, family='Arial', color='black'),
    xaxis_title='Score',
    yaxis_title='Livros',
    xaxis=dict(
        title_font=dict(size=16, family='Arial', color='black'),
        tickfont=dict(size=12)
    ),
    yaxis=dict(
        title_font=dict(size=16, family='Arial', color='black'),
        tickfont=dict(size=12)
    ),
    height=800,  # Aumentar a altura do gráfico
    width=1300,
    plot_bgcolor='white',  # Fundo do gráfico branco
    paper_bgcolor='white',  # Fundo do papel branco
    margin=dict(l=150, r=50, t=50, b=50)  # Margens
)

fig.show()

In [61]:
bd_sorted[['Title', 'categories', 'score']][:10]

,Title,categories,score
61163,A Searching Heart (Prairie Legacy Series #2),Fiction,4.80
42731,Joni,Biography & Autobiography,4.68
23055,Christianity and liberalism,Christianity,4.66
14923,Population: 485 : Meeting Your Neighbors One S...,Travel,4.62
97126,Constitutional Law: Principles and Policies (A...,Law,4.62
154230,Ramona the pest,Juvenile Fiction,4.61
34985,Judas Child,Fiction,4.58
122371,More Wealth Without Risk,"Finance, Personal",4.57
140003,Adobe Photoshop CS2 One-On-One,Computers,4.56
162907,Alaska (Turtleback School & Library Binding Ed...,Young Adult Nonfiction,4.54


In [62]:
bd_sorted[['Title', 'categories', 'score']][:10].to_parquet(r'..\\..\\Desafio_A3Data\\Databases\\top_books_maio.parquet', engine='pyarrow')

In [53]:
# Dados para o gráfico de barras com a variável 'publisher' no eixo y
categorias_editora = bd['publisher'].unique()
valores_editora = bd.groupby('publisher')['score'].mean()

# Ordenar os dados de scores médios por editora do maior para o menor
valores_editora_sorted = valores_editora.sort_values(ascending=False)

# Criação do gráfico de barras horizontal com a variável 'publisher' no eixo y
fig2 = go.Figure(data=[
    go.Bar(
        name='Valores', 
        y=valores_editora_sorted.index, 
        x=valores_editora_sorted, 
        orientation='h',
        marker=dict(
            color='#1f77b4'  # Cor das barras
        ),
        text=valores_editora_sorted.round(2),  # Adiciona os valores como rótulos arredondados
        textposition='outside',  # Posição dos rótulos
        hoverinfo='y+x'  # Informação exibida ao passar o mouse
    )
])

# Personalização do layout do gráfico com a variável 'publisher' no eixo y
fig2.update_layout(
    title='Scores por Editora: Avaliação dos Leitores',
    title_font=dict(size=20, family='Arial', color='black'),
    xaxis_title='Score',
    yaxis_title='Editora',
    xaxis=dict(
        title_font=dict(size=16, family='Arial', color='black'),
        tickfont=dict(size=12)
    ),
    yaxis=dict(
        title_font=dict(size=16, family='Arial', color='black'),
        tickfont=dict(size=12)
    ),
    height=700,  # Aumentar a altura do gráfico
    width=800,   # Ajustar a largura do gráfico
    plot_bgcolor='white',  # Fundo do gráfico branco
    paper_bgcolor='white',  # Fundo do papel branco
    margin=dict(l=150, r=50, t=50, b=50)  # Margens
)

fig2.show()

In [54]:
valores_editora_sorted[:5].reset_index()

,publisher,score
0,Bethany House,4.80
1,Zondervan,4.68
2,Harper Perennial,4.62
3,Aspen Publishers,4.62
4,Simon & Schuster,4.57


In [63]:
valores_editora_sorted[:5].reset_index().to_parquet(r'..\\..\\Desafio_A3Data\\Databases\\top_publishers_maio.parquet', engine='pyarrow')

In [55]:
# Dados para o gráfico de barras com a variável 'categories' no eixo y
categorias_segmento = bd['categories'].unique()
valores_segmento = bd.groupby('categories')['score'].mean()

# Ordenar os dados de scores médios por segmento do maior para o menor
valores_segmento_sorted = valores_segmento.sort_values(ascending=False)

# Criação do gráfico de barras horizontal com a variável 'categories' no eixo y
fig3 = go.Figure(data=[
    go.Bar(
        name='Valores', 
        y=valores_segmento_sorted.index, 
        x=valores_segmento_sorted, 
        orientation='h',
        marker=dict(
            color='#1f77b4'  # Cor das barras
        ),
        text=valores_segmento_sorted.round(2),  # Adiciona os valores como rótulos arredondados
        textposition='outside',  # Posição dos rótulos
        hoverinfo='y+x'  # Informação exibida ao passar o mouse
    )
])

# Personalização do layout do gráfico com a variável 'categories' no eixo y
fig3.update_layout(
    title='Scores por Categoria: Avaliação dos Leitores',
    title_font=dict(size=20, family='Arial', color='black'),
    xaxis_title='Score',
    yaxis_title='Categoria',
    xaxis=dict(
        title_font=dict(size=16, family='Arial', color='black'),
        tickfont=dict(size=12),
        range=[0, 5],  # Definir a faixa do eixo x
        dtick=1  # Definir o espaçamento entre os ticks do eixo x
    ),
    yaxis=dict(
        title_font=dict(size=16, family='Arial', color='black'),
        tickfont=dict(size=12)
    ),
    height=600,  # Ajustar a altura do gráfico
    width=1000,   # Ajustar a largura do gráfico
    plot_bgcolor='white',  # Fundo do gráfico branco
    paper_bgcolor='white',  # Fundo do papel branco
    margin=dict(l=150, r=50, t=50, b=50)  # Margens
)

# Exibir o gráfico com a variável 'categories' no eixo y
fig3.show()

In [56]:
valores_segmento_sorted[:5].reset_index()

,categories,score
0,Christianity,4.66
1,Law,4.62
2,Travel,4.62
3,"Finance, Personal",4.57
4,Biography & Autobiography,4.56


In [65]:
valores_segmento_sorted[:5].reset_index().to_parquet(r'..\\..\\Desafio_A3Data\\Databases\\top_categories_maio.parquet', engine='pyarrow')

In [57]:
# Dados para o gráfico de barras com a variável 'authors' no eixo y
categorias_autor = bd['authors'].unique()
valores_autor = bd.groupby('authors')['score'].mean()

# Ordenar os dados de scores médios por autor do maior para o menor
valores_autor_sorted = valores_autor.sort_values(ascending=False)

# Criação do gráfico de barras horizontal com a variável 'authors' no eixo y
fig4 = go.Figure(data=[
    go.Bar(
        name='Valores', 
        y=valores_autor_sorted.index, 
        x=valores_autor_sorted, 
        orientation='h',
        marker=dict(
            color='#1f77b4'  # Cor das barras
        ),
        text=valores_autor_sorted.round(2),  # Adiciona os valores como rótulos arredondados
        textposition='outside',  # Posição dos rótulos
        hoverinfo='y+x'  # Informação exibida ao passar o mouse
    )
])

# Personalização do layout do gráfico com a variável 'authors' no eixo y
fig4.update_layout(
    title='Scores por Autor(a): Avaliação dos Leitores',
    title_font=dict(size=20, family='Arial', color='black'),
    xaxis_title='Score',
    yaxis_title='Autor(a)',
    xaxis=dict(
        title_font=dict(size=16, family='Arial', color='black'),
        tickfont=dict(size=12),
        range=[0, 5],  # Definir a faixa do eixo x
        dtick=0.5  # Definir o espaçamento entre os ticks do eixo x
    ),
    yaxis=dict(
        title_font=dict(size=16, family='Arial', color='black'),
        tickfont=dict(size=12)
    ),
    height=800,  # Ajustar a altura do gráfico
    width=1000,   # Ajustar a largura do gráfico
    plot_bgcolor='white',  # Fundo do gráfico branco
    paper_bgcolor='white',  # Fundo do papel branco
    margin=dict(l=150, r=50, t=50, b=50)  # Margens
)

# Exibir o gráfico com a variável 'authors' no eixo y
fig4.show()

In [58]:
valores_autor_sorted[:5].reset_index()

,authors,score
0,Janette Oke,4.80
1,Joni Eareckson Tada,4.68
2,John Gresham Machen,4.66
3,Erwin Chemerinsky,4.62
4,Michael Perry,4.62


In [66]:
valores_autor_sorted[:5].reset_index().to_parquet(r'..\\..\\Desafio_A3Data\\Databases\\top_authors_maio.parquet', engine='pyarrow')

#### Hipóteses: Autor mais bem avaliado é o mais lido


In [11]:
bd['authors'].nunique()

38

In [9]:
result = bd[['authors', 'score']].groupby('authors').agg(
    mean_score=('score', 'mean'),
    count=('authors', 'size')
).reset_index()

result = result.sort_values(by='mean_score', ascending=False)
result

,authors,mean_score,count
15,Janette Oke,4.80,1
21,Joni Eareckson Tada,4.68,1
17,John Gresham Machen,4.66,1
11,Erwin Chemerinsky,4.62,1
26,Michael Perry,4.62,1
3,Beverly Cleary,4.61,1
5,Carol OConnell,4.58,1
7,Charles J. Givens,4.57,1
9,Deke McClelland,4.56,1
1,"Arthur Bochner, Rose Bochner, Adriane G. Berg",4.54,1


#### Hipótese: Existem leitores problemáticos 

In [48]:
result1 = br[['User_id', 'score']].groupby('User_id').agg(
    mean_score=('score', 'mean'),
    count=('User_id', 'size')
).reset_index()

result1 = result1.sort_values(by=['mean_score', 'count'], ascending=[True, False])
result1.head(10)

,User_id,mean_score,count
141,A172AW5IS4Z79Y,1.0,4
1041,A2F0ETLPN3JFLO,1.0,4
696,A1Y87XPULGDR37,1.0,3
1295,A2RJ1N332RBDDJ,1.0,3
110,A15W834OW4GEFW,1.0,2
394,A1K18REXZ9TRAW,1.0,2
407,A1KNTHDRNK3DCX,1.0,2
511,A1PE9PCGRBGB37,1.0,2
516,A1PN8PSFV99Q41,1.0,2
1380,A2W1T6WAAVEZRC,1.0,2


In [21]:
br[(br['User_id'] == 'A172AW5IS4Z79Y') | (br['User_id'] == 'A2F0ETLPN3JFLO')]['profileName'].unique()

array(['Spaniardx', 'Shawn M. Owen "solocolo"'], dtype=object)

#### Hipótese: Existem leitores ecléticos

In [47]:
result2 = br[['User_id', 'score']].groupby('User_id').agg(
    count=('User_id', 'size')
).reset_index()

result2 = result2.sort_values(by=['count'], ascending=[False])
result2[['User_id', 'count']].head(10)

,User_id,count
415,A1L43KWWR05PCS,7
2156,AA0BB36HOBYLC,6
952,A2AYSFGUP5VTY3,5
15,A114YQ7ZT9Y1W5,5
395,A1K1JW1C5CUSUZ,5
2543,ATTLUK81FJVJ3,5
1917,A3PM7CTXMNLMBC,4
362,A1I78HZLE3O1SD,4
1509,A32GGHNOVN6ASX,4
1428,A2YIUQF3LKRU7F,4


In [36]:
cutup = br[(br['User_id'] == 'A1L43KWWR05PCS') | (br['User_id'] == 'AA0BB36HOBYLC')][['profileName', 'Title']]
list1 = cutup[cutup['profileName'] == 'Lawyeraau']['Title'].unique()
list2 = cutup[cutup['profileName'] == 'Deborah Dieleman "Jen D"']['Title'].unique()

In [40]:
#Lawyeraau
bd[bd['Title'].isin(list1)][['Title', 'categories']]

,Title,categories
34985,Judas Child,Fiction
130675,Don't Pee on My Leg and Tell Me It's Raining: ...,Biography & Autobiography
158148,Member of the Wedding,Fiction


In [41]:
#Deborah Dieleman "Jen D"
bd[bd['Title'].isin(list2)][['Title', 'categories']]

,Title,categories
61163,A Searching Heart (Prairie Legacy Series #2),Fiction
117711,Five Little Peppers and How They Grew,Brothers and sisters


#### Hipóteses: Livros populares com baixa avaliação; Livro mais bem avaliado é o mais lido

In [46]:
result3 = bd[['Title', 'score']].groupby('Title').agg(
    mean_score=('score', 'mean'),
    count=('Title', 'size')
).reset_index()

result3 = result3.sort_values(by=['mean_score', 'count'], ascending=[True, False])
result3.head(10)

,Title,mean_score,count
10,Elementary Differential Equations and Boundary...,3.02,1
31,The Geographer's Library,3.06,1
21,Member of the Wedding,3.28,1
26,Splinter of the Mind's Eye,3.50,1
14,Haunts of the Bushrangers,3.57,1
3,Abraham,3.67,1
37,The Wapshot Chronicle,3.75,1
35,The Ten Thousand: A Novel of Ancient Greece,3.79,1
12,Everyday Greatness: Inspiration for a Meaningf...,3.88,1
19,"Look Homeward, Angel",3.90,1


In [50]:
result4 = bd[['Title', 'score']].groupby('Title').agg(
    mean_score=('score', 'mean'),
    count=('Title', 'size')
).reset_index()

result4 = result4.sort_values(by=['mean_score', 'count'], ascending=[False, False])
result4.head(10)

,Title,mean_score,count
2,A Searching Heart (Prairie Legacy Series #2),4.80,1
16,Joni,4.68,1
6,Christianity and liberalism,4.66,1
7,Constitutional Law: Principles and Policies (A...,4.62,1
24,Population: 485 : Meeting Your Neighbors One S...,4.62,1
25,Ramona the pest,4.61,1
17,Judas Child,4.58,1
22,More Wealth Without Risk,4.57,1
4,Adobe Photoshop CS2 One-On-One,4.56,1
5,Alaska (Turtleback School & Library Binding Ed...,4.54,1
